In [4]:
import argparse
import gym
import numpy as np
from itertools import count

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical



env = gym.make('CartPole-v0')
#env.seed(args.seed)
#torch.manual_seed(args.seed)


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.affine1 = nn.Linear(4, 128)
        self.affine2 = nn.Linear(128, 2)

        self.saved_log_probs = []
        self.rewards = []

    def forward(self, x):
        x = F.relu(self.affine1(x))
        action_scores = self.affine2(x)
        return F.softmax(action_scores, dim=1)


policy = Policy()
optimizer = optim.Adam(policy.parameters(), lr=1e-2)
eps = np.finfo(np.float32).eps.item()


def select_action(state):
    state = torch.from_numpy(state).float().unsqueeze(0)
    probs = policy(state)
    m = Categorical(probs)
    action = m.sample()
    policy.saved_log_probs.append(m.log_prob(action))
    return action.item()


def finish_episode():
    R = 0
    policy_loss = []
    rewards = []
    gamma = .5
    for r in policy.rewards[::-1]:
        R = r + gamma * R
        rewards.insert(0, R)
    rewards = torch.tensor(rewards)
    rewards = (rewards - rewards.mean()) / (rewards.std() + eps)
    for log_prob, reward in zip(policy.saved_log_probs, rewards):
        policy_loss.append(-log_prob * reward)
    optimizer.zero_grad()
    policy_loss = torch.cat(policy_loss).sum()
    policy_loss.backward()
    optimizer.step()
    del policy.rewards[:]
    del policy.saved_log_probs[:]


def main():
    running_reward = 10
    for i_episode in count(1):
        state = env.reset()
        for t in range(10000):  # Don't infinite loop while learning
            action = select_action(state)
            state, reward, done, _ = env.step(action)
      
            policy.rewards.append(reward)
            if done:
                break

        running_reward = running_reward * 0.99 + t * 0.01
        finish_episode()
        
        
        log_interval = 10
        
        if i_episode % log_interval == 0:
            print('Episode {}\tLast length: {:5d}\tAverage length: {:.2f}'.format(
                i_episode, t, running_reward))
        if running_reward > env.spec.reward_threshold:
            print("Solved! Running reward is now {} and "
                  "the last episode runs to {} time steps!".format(running_reward, t))
            break


if __name__ == '__main__':
    main()

Episode 10	Last length:    21	Average length: 11.14
Episode 20	Last length:     9	Average length: 11.11
Episode 30	Last length:     8	Average length: 10.87
Episode 40	Last length:     8	Average length: 10.81
Episode 50	Last length:     8	Average length: 10.69
Episode 60	Last length:    13	Average length: 10.66
Episode 70	Last length:     8	Average length: 10.47
Episode 80	Last length:    10	Average length: 10.36
Episode 90	Last length:     9	Average length: 10.24
Episode 100	Last length:     8	Average length: 10.12
Episode 110	Last length:     9	Average length: 10.13
Episode 120	Last length:     7	Average length: 10.06
Episode 130	Last length:     9	Average length: 10.05
Episode 140	Last length:     8	Average length: 9.96
Episode 150	Last length:     8	Average length: 9.81
Episode 160	Last length:    12	Average length: 9.88
Episode 170	Last length:    13	Average length: 9.92
Episode 180	Last length:    14	Average length: 10.34
Episode 190	Last length:    47	Average length: 12.71
Episod

KeyboardInterrupt: 